In [15]:
import os
import sys
import argparse
import re
import augment_process_dataset as apd
import torch
import numpy as np
import pandas as pd
import unicodedata
#import entity_detection as ed
#from test_main import entity_predict
from nltk.corpus import stopwords
from itertools import compress
from evaluation import evaluation, get_span
from argparse import ArgumentParser
from torchtext import data
from sklearn.metrics.pairwise import euclidean_distances
from fuzzywuzzy import fuzz
from util import www2fb, processed_text, clean_uri

In [16]:
class Args:
    seed = 345
    cuda = False
    gpu = -1
    embed_dim = 250
    batch_size = 1
    dete_model = 'dete_best_model.pt'
    entity_model = 'entity_best_model.pt'
    pred_model='pred_best_model.pt'
    output='preprocess'
    data = './data/penn'
    model = 'LSTM'
    emsize = 200
    nhid = 200
args=Args()

In [17]:
TEXT = data.Field(lower=True)
ED = data.Field()
ip = input('enter a question: ')
ip = processed_text(ip)

enter a question: who is daft punk?


In [18]:
outfile = open(os.path.join(args.output, 'input_file.txt'), 'w')
#with open(os.path.join(args.output, 'freebase-FB5M.txt'), "r") as f:
#    searchlines = f.readlines()
#for i, line in enumerate(searchlines):
#    if "india" in line: 
#        for l in searchlines[i:i+3]:
#            print (l)
ed=apd.reverseLinking(ip,None)
ed = ed[1]
outfile.write('{}\t{}\t\n'.format(ip, ed))
outfile.close()

In [19]:
out= data.TabularDataset(path=os.path.join(args.output, 'input_file.txt'), format='tsv', fields=[('text', TEXT), ('ed', ED)])

In [20]:
def entity_predict(dataset_iter):
    model.eval()
    dataset_iter.init_epoch()
    gold_list = []
    pred_list = []
    dete_result = []
    question_list = []
    for data_batch_idx, data_batch in enumerate(dataset_iter):
        batch_size = 1#data_batch.text.size()[1]
        answer = torch.max(model(data_batch), 1)[1].view(data_batch.ed.size())
        answer[(data_batch.text.data == 1)] = 1
        answer = np.transpose(answer.cpu().data.numpy())
        gold_list.append(np.transpose(data_batch.ed.cpu().data.numpy()))
        index_question = np.transpose(data_batch.text.cpu().data.numpy())
        question_array = index2word[index_question]
        dete_result.extend(answer)
        question_list.extend(question_array)
        #for i in range(batch_size):  # If no word is detected as entity, select top 3 possible words
        #    if all([j == 1 or j == idxO for j in answer[i]]):
        #        index = list(range(i, scores.shape[0], batch_size))
        #        FindOidx = [j for j, x in enumerate(answer[i]) if x == idxO]
        #        idx_in_socres = [index[j] for j in FindOidx]
        #        subscores = scores[idx_in_socres]
        #        answer[i][torch.sort(torch.max(subscores, 1)[0], descending=True)[1][0:min(2, len(FindOidx))]] = idxI
        pred_list.append(answer)
    return dete_result, question_list

In [21]:
train = data.TabularDataset(path=os.path.join(args.output, 'dete_train.txt'), format='tsv', fields=[('text', TEXT), ('ed', ED)])
field = [('id', None), ('sub', None), ('entity', None), ('relation', None), ('obj', None), ('text', TEXT), ('ed', ED)]

In [22]:
dev, test = data.TabularDataset.splits(path=args.output, validation='valid.txt', test='test.txt', format='tsv', fields=field)
TEXT.build_vocab(train, dev, test)
ED.build_vocab(train, dev)
total_num = len(test)
print('total num of example: {}'.format(total_num))

total num of example: 21687


In [23]:
# load the model
if args.gpu == -1: # Load all tensors onto the CPU
    test_iter = data.Iterator(out, batch_size=args.batch_size, train=False, repeat=False, sort=False, shuffle=False, 
                              sort_within_batch=False)
    model = torch.load(args.dete_model, map_location=lambda storage, loc: storage)
    model.config.cuda = False
else:
    test_iter = data.Iterator(test, batch_size=args.batch_size, device=torch.device('cuda', args.gpu), train=False,
                              repeat=False, sort=False, shuffle=False, sort_within_batch=False)
    model = torch.load(args.dete_model, map_location=lambda storage, loc: storage.cuda(args.gpu))
index2tag = np.array(ED.vocab.itos)
idxO = int(np.where(index2tag == 'O')[0][0])  # Index for 'O'
idxI = int(np.where(index2tag == 'I')[0][0])  # Index for 'I'
index2word = np.array(TEXT.vocab.itos)
# run the model on the test set and write the output to a file
dete_result, question_list= entity_predict(dataset_iter=test_iter)
print(question_list)
del model

[array(['who', 'is', '<unk>', 'punk', '?'], dtype='<U62')]


In [24]:
realed=[]
for i in range(len(dete_result[0])):
    if dete_result[0][i] == 2:
        realed.append('O')
    if dete_result[0][i] == 3:
        realed.append('I')
    realed.append(' ')
del realed[-1]
realed=''.join(realed)  
outfile = open(os.path.join(args.output, 'input_file.txt'), 'w')
outfile.write('{}\t{}\t\n'.format(ip, realed))
outfile.close()

In [25]:
'''w_filenameTSV = './preprocess/mid2name.tsv'
tsv_read = pd.read_csv(w_filenameTSV, sep='\t')'''

"w_filenameTSV = './preprocess/mid2name.tsv'\ntsv_read = pd.read_csv(w_filenameTSV, sep='\t')"

In [26]:
'''outfile = open(os.path.join(args.output, 'heads_toes.txt'), 'w')
mid_cid = []
for i in range(len(tsv_read.mid)):
    mid_cid.append(www2fb(tsv_read.mid[i]))
    mid_cid[i] = mid_cid[i].replace('m0','m.0')
    outfile.write('{}\t{}\t\n'.format(mid_cid[i],tsv_read.name[i]))
outfile.close()'''

"outfile = open(os.path.join(args.output, 'heads_toes.txt'), 'w')\nmid_cid = []\nfor i in range(len(tsv_read.mid)):\n    mid_cid.append(www2fb(tsv_read.mid[i]))\n    mid_cid[i] = mid_cid[i].replace('m0','m.0')\n    outfile.write('{}\t{}\t\n'.format(mid_cid[i],tsv_read.name[i]))\noutfile.close()"

In [27]:
'''f = open("./preprocess/heads_toes.txt", "r")
print(f.read(100))'''

'f = open("./preprocess/heads_toes.txt", "r")\nprint(f.read(100))'

In [28]:
'''from rdflib import Graph
g = Graph()
g.parse("demo.nt", format="nt")

len(g) # prints 2

import pprint
for stmt in g:
    pprint.pprint(stmt)

# prints :
(rdflib.term.URIRef('http://bigasterisk.com/foaf.rdf#drewp'),
 rdflib.term.URIRef('http://example.com/says'),
 rdflib.term.Literal(u'Hello world'))
(rdflib.term.URIRef('http://bigasterisk.com/foaf.rdf#drewp'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://xmlns.com/foaf/0.1/Person'))
 
 
 
 from wikidata.client import Client
>>> client = Client()  # doctest: +SKIP
>>> entity = client.get('Q20145', load=True)
>>> entity
 '''

'from rdflib import Graph\ng = Graph()\ng.parse("demo.nt", format="nt")\n\nlen(g) # prints 2\n\nimport pprint\nfor stmt in g:\n    pprint.pprint(stmt)\n\n# prints :\n(rdflib.term.URIRef(\'http://bigasterisk.com/foaf.rdf#drewp\'),\n rdflib.term.URIRef(\'http://example.com/says\'),\n rdflib.term.Literal(u\'Hello world\'))\n(rdflib.term.URIRef(\'http://bigasterisk.com/foaf.rdf#drewp\'),\n rdflib.term.URIRef(\'http://www.w3.org/1999/02/22-rdf-syntax-ns#type\'),\n rdflib.term.URIRef(\'http://xmlns.com/foaf/0.1/Person\'))\n \n \n \n from wikidata.client import Client\n>>> client = Client()  # doctest: +SKIP\n>>> entity = client.get(\'Q20145\', load=True)\n>>> entity\n '